In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import random
from math import pi, floor
import math
import typing

import torch
import torch.nn as nn
from torch.autograd.gradcheck import zero_gradients
import torch.nn.functional as F
import torch.autograd as autograd
import torchvision.models as models
from PIL import Image
from torchvision import transforms
import requests, io
from torch.autograd import Variable
import scipy.misc
from torch.fft import ifft, fft, ifftn, fftn
from IQA_pytorch import SSIM, VIF

In [ ]:
torch.autograd.set_detect_anomaly(True)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
name = 'name_of_image.png'
name2 = 'name_of_image.xlsx'

filepath = 'image/' + name
adv_filepath = 'result/real/' # + "name_of_model/" + name
adv_filepath2 = 'result/real/' # + "name_of_model/" + name2

In [ ]:
def fft2(data):
    assert data.size(-1) == 2
    data = torch.fft(data, 2, normalized=True)
    return data


def fft2c(data):
    """
    Apply centered 2 dimensional Fast Fourier Transform.
    Args:
        data (torch.Tensor): Complex valued input data containing at least 3 dimensions: dimensions
            -3 & -2 are spatial dimensions and dimension -1 has size 2. All other dimensions are
            assumed to be batch dimensions.
    Returns:
        torch.Tensor: The FFT of the input.
    """
    assert data.size(-1) == 2
    data = ifftshift(data, dim=(-3, -2))
    data = torch.fft(data, 2, normalized=True)
    data = fftshift(data, dim=(-3, -2))
    return data


def ifft2(data):
    assert data.size(-1) == 2
    data = torch.ifft(data, 2, normalized=True)
    return data


def ifft2c(data):
    """
    Apply centered 2-dimensional Inverse Fast Fourier Transform.
    Args:
        data (torch.Tensor): Complex valued input data containing at least 3 dimensions: dimensions
            -3 & -2 are spatial dimensions and dimension -1 has size 2. All other dimensions are
            assumed to be batch dimensions.
    Returns:
        torch.Tensor: The IFFT of the input.
    """
    assert data.size(-1) == 2
    data = ifftshift(data, dim=(-3, -2))
    data = torch.ifft(data, 2, normalized=True)
    data = fftshift(data, dim=(-3, -2))
    return data


def roll(x, shift, dim):
    """
    Similar to np.roll but applies to PyTorch Tensors
    """
    if isinstance(shift, (tuple, list)):
        assert len(shift) == len(dim)
        for s, d in zip(shift, dim):
            x = roll(x, s, d)
        return x
    shift = shift % x.size(dim)
    if shift == 0:
        return x
    left = x.narrow(dim, 0, x.size(dim) - shift)
    right = x.narrow(dim, x.size(dim) - shift, shift)
    return torch.cat((right, left), dim=dim)


def fftshift(x, dim=None):
    """
    Similar to np.fft.fftshift but applies to PyTorch Tensors
    """
    if dim is None:
        dim = tuple(range(x.dim()))
        shift = [dim // 2 for dim in x.shape]
    elif isinstance(dim, int):
        shift = x.shape[dim] // 2
    else:
        shift = [x.shape[i] // 2 for i in dim]
    return roll(x, shift, dim)


def ifftshift(x, dim=None):
    """
    Similar to np.fft.ifftshift but applies to PyTorch Tensors
    """
    if dim is None:
        dim = tuple(range(x.dim()))
        shift = [(dim + 1) // 2 for dim in x.shape]
    elif isinstance(dim, int):
        shift = (x.shape[dim] + 1) // 2
    else:
        shift = [(x.shape[i] + 1) // 2 for i in dim]
    return roll(x, shift, dim)


In [ ]:
__all__ = ['imresize'] 

_I = typing.Optional[int]
_D = typing.Optional[torch.dtype]

def nearest_contribution(x: torch.Tensor) -> torch.Tensor:
    range_around_0 = torch.logical_and(x.gt(-0.5), x.le(0.5))
    cont = range_around_0.to(dtype=x.dtype)
    return cont

def linear_contribution(x: torch.Tensor) -> torch.Tensor:
    ax = x.abs()
    range_01 = ax.le(1)
    cont = (1 - ax) * range_01.to(dtype=x.dtype)
    return cont

def cubic_contribution(x: torch.Tensor, a: float=-0.5) -> torch.Tensor:
    ax = x.abs()
    ax2 = ax * ax
    ax3 = ax * ax2

    range_01 = ax.le(1)
    range_12 = torch.logical_and(ax.gt(1), ax.le(2))

    cont_01 = (a + 2) * ax3 - (a + 3) * ax2 + 1
    cont_01 = cont_01 * range_01.to(dtype=x.dtype)

    cont_12 = (a * ax3) - (5 * a * ax2) + (8 * a * ax) - (4 * a)
    cont_12 = cont_12 * range_12.to(dtype=x.dtype)

    cont = cont_01 + cont_12
    return cont

def gaussian_contribution(x: torch.Tensor, sigma: float=2.0) -> torch.Tensor:
    range_3sigma = (x.abs() <= 3 * sigma + 1)
    # Normalization will be done after
    cont = torch.exp(-x.pow(2) / (2 * sigma**2))
    cont = cont * range_3sigma.to(dtype=x.dtype)
    return cont

def discrete_kernel(
        kernel: str, scale: float, antialiasing: bool=True) -> torch.Tensor:

    '''
    For downsampling with integer scale only.
    '''
    downsampling_factor = int(1 / scale)
    if kernel == 'cubic':
        kernel_size_orig = 4
    else:
        raise ValueError('Pass!')

    if antialiasing:
        kernel_size = kernel_size_orig * downsampling_factor
    else:
        kernel_size = kernel_size_orig

    if downsampling_factor % 2 == 0:
        a = kernel_size_orig * (0.5 - 1 / (2 * kernel_size))
    else:
        kernel_size -= 1
        a = kernel_size_orig * (0.5 - 1 / (kernel_size + 1))

    with torch.no_grad():
        r = torch.linspace(-a, a, steps=kernel_size)
        k = cubic_contribution(r).view(-1, 1)
        k = torch.matmul(k, k.t())
        k /= k.sum()

    return k

def reflect_padding(
        x: torch.Tensor,
        dim: int,
        pad_pre: int,
        pad_post: int) -> torch.Tensor:

    b, c, h, w = x.size()
    if dim == 2 or dim == -2:
        padding_buffer = x.new_zeros(b, c, h + pad_pre + pad_post, w)
        padding_buffer[..., pad_pre:(h + pad_pre), :].copy_(x)
        for p in range(pad_pre):
            padding_buffer[..., pad_pre - p - 1, :].copy_(x[..., p, :])
        for p in range(pad_post):
            padding_buffer[..., h + pad_pre + p, :].copy_(x[..., -(p + 1), :])
    else:
        padding_buffer = x.new_zeros(b, c, h, w + pad_pre + pad_post)
        padding_buffer[..., pad_pre:(w + pad_pre)].copy_(x)
        for p in range(pad_pre):
            padding_buffer[..., pad_pre - p - 1].copy_(x[..., p])
        for p in range(pad_post):
            padding_buffer[..., w + pad_pre + p].copy_(x[..., -(p + 1)])

    return padding_buffer

def padding(
        x: torch.Tensor,
        dim: int,
        pad_pre: int,
        pad_post: int,
        padding_type: typing.Optional[str]='reflect') -> torch.Tensor:

    if padding_type is None:
        return x
    elif padding_type == 'reflect':
        x_pad = reflect_padding(x, dim, pad_pre, pad_post)
    else:
        raise ValueError('{} padding is not supported!'.format(padding_type))

    return x_pad

def get_padding(
        base: torch.Tensor,
        kernel_size: int,
        x_size: int) -> typing.Tuple[int, int, torch.Tensor]:

    base = base.long()
    r_min = base.min()
    r_max = base.max() + kernel_size - 1

    if r_min <= 0:
        pad_pre = -r_min
        pad_pre = pad_pre.item()
        base += pad_pre
    else:
        pad_pre = 0

    if r_max >= x_size:
        pad_post = r_max - x_size + 1
        pad_post = pad_post.item()
    else:
        pad_post = 0

    return pad_pre, pad_post, base

def get_weight(
        dist: torch.Tensor,
        kernel_size: int,
        kernel: str='cubic',
        sigma: float=2.0,
        antialiasing_factor: float=1) -> torch.Tensor:

    buffer_pos = dist.new_zeros(kernel_size, len(dist))
    for idx, buffer_sub in enumerate(buffer_pos):
        buffer_sub.copy_(dist - idx)

    # Expand (downsampling) / Shrink (upsampling) the receptive field.
    buffer_pos *= antialiasing_factor
    if kernel == 'cubic':
        weight = cubic_contribution(buffer_pos)
    elif kernel == 'gaussian':
        weight = gaussian_contribution(buffer_pos, sigma=sigma)
    else:
        raise ValueError('{} kernel is not supported!'.format(kernel))

    weight /= weight.sum(dim=0, keepdim=True)
    return weight

def reshape_tensor(x: torch.Tensor, dim: int, kernel_size: int) -> torch.Tensor:
    # Resize height
    if dim == 2 or dim == -2:
        k = (kernel_size, 1)
        h_out = x.size(-2) - kernel_size + 1
        w_out = x.size(-1)
    # Resize width
    else:
        k = (1, kernel_size)
        h_out = x.size(-2)
        w_out = x.size(-1) - kernel_size + 1

    unfold = F.unfold(x, k)
    unfold = unfold.view(unfold.size(0), -1, h_out, w_out)
    return unfold

def reshape_input(
        x: torch.Tensor) -> typing.Tuple[torch.Tensor, _I, _I, _I, _I]:

    if x.dim() == 4:
        b, c, h, w = x.size()
    elif x.dim() == 3:
        c, h, w = x.size()
        b = None
    elif x.dim() == 2:
        h, w = x.size()
        b = c = None
    else:
        raise ValueError('{}-dim Tensor is not supported!'.format(x.dim()))

    x = x.view(-1, 1, h, w)
    return x, b, c, h, w

def reshape_output(
        x: torch.Tensor, b: _I, c: _I) -> torch.Tensor:

    rh = x.size(-2)
    rw = x.size(-1)
    # Back to the original dimension
    if b is not None:
        x = x.view(b, c, rh, rw)        # 4-dim
    else:
        if c is not None:
            x = x.view(c, rh, rw)       # 3-dim
        else:
            x = x.view(rh, rw)          # 2-dim

    return x

def cast_input(x: torch.Tensor) -> typing.Tuple[torch.Tensor, _D]:
    if x.dtype != torch.float32 or x.dtype != torch.float64:
        dtype = x.dtype
        x = x.float()
    else:
        dtype = None

    return x, dtype

def cast_output(x: torch.Tensor, dtype: _D) -> torch.Tensor:
    if dtype is not None:
        if not dtype.is_floating_point:
            x = x.round()
        # To prevent over/underflow when converting types
        if dtype is torch.uint8:
            x = x.clamp(0, 255)

        x = x.to(dtype=dtype)

    return x

def resize_1d(
        x: torch.Tensor,
        dim: int,
        size: typing.Optional[int],
        scale: typing.Optional[float],
        kernel: str='cubic',
        sigma: float=2.0,
        padding_type: str='reflect',
        antialiasing: bool=True) -> torch.Tensor:

    # Identity case
    if scale == 1:
        return x

    # Default bicubic kernel with antialiasing (only when downsampling)
    if kernel == 'cubic':
        kernel_size = 4
    else:
        kernel_size = math.floor(6 * sigma)

    if antialiasing and (scale < 1):
        antialiasing_factor = scale
        kernel_size = math.ceil(kernel_size / antialiasing_factor)
    else:
        antialiasing_factor = 1

    # We allow margin to both sizes
    kernel_size += 2

    # Weights only depend on the shape of input and output,
    # so we do not calculate gradients here.
    with torch.no_grad():
        pos = torch.linspace(
            0, size - 1, steps=size, dtype=x.dtype, device=x.device,
        )
        pos = (pos + 0.5) / scale - 0.5
        base = pos.floor() - (kernel_size // 2) + 1
        dist = pos - base
        weight = get_weight(
            dist,
            kernel_size,
            kernel=kernel,
            sigma=sigma,
            antialiasing_factor=antialiasing_factor,
        )
        pad_pre, pad_post, base = get_padding(base, kernel_size, x.size(dim))

    # To backpropagate through x
    x_pad = padding(x, dim, pad_pre, pad_post, padding_type=padding_type)
    unfold = reshape_tensor(x_pad, dim, kernel_size)
    # Subsampling first
    if dim == 2 or dim == -2:
        sample = unfold[..., base, :]
        weight = weight.view(1, kernel_size, sample.size(2), 1)
    else:
        sample = unfold[..., base]
        weight = weight.view(1, kernel_size, 1, sample.size(3))

    # Apply the kernel
    x = sample * weight
    x = x.sum(dim=1, keepdim=True)
    return x

def downsampling_2d(
        x: torch.Tensor,
        k: torch.Tensor,
        scale: int,
        padding_type: str='reflect') -> torch.Tensor:

    c = x.size(1)
    k_h = k.size(-2)
    k_w = k.size(-1)

    k = k.to(dtype=x.dtype, device=x.device)
    k = k.view(1, 1, k_h, k_w)
    k = k.repeat(c, c, 1, 1)
    e = torch.eye(c, dtype=k.dtype, device=k.device, requires_grad=False)
    e = e.view(c, c, 1, 1)
    k = k * e

    pad_h = (k_h - scale) // 2
    pad_w = (k_w - scale) // 2
    x = padding(x, -2, pad_h, pad_h, padding_type=padding_type)
    x = padding(x, -1, pad_w, pad_w, padding_type=padding_type)
    y = F.conv2d(x, k, padding=0, stride=scale)
    return y

def imresize(
        x: torch.Tensor,
        scale: typing.Optional[float]=None,
        sizes: typing.Optional[typing.Tuple[int, int]]=None,
        kernel: typing.Union[str, torch.Tensor]='cubic',
        sigma: float=2,
        rotation_degree: float=0,
        padding_type: str='reflect',
        antialiasing: bool=True) -> torch.Tensor:


    if scale is None and sizes is None:
        raise ValueError('One of scale or sizes must be specified!')
    if scale is not None and sizes is not None:
        raise ValueError('Please specify scale or sizes to avoid conflict!')

    x, b, c, h, w = reshape_input(x)

    if sizes is None:

        # Determine output size
        sizes = (math.ceil(h * scale), math.ceil(w * scale))
        scales = (scale, scale)

    if scale is None:
        scales = (sizes[0] / h, sizes[1] / w)

    x, dtype = cast_input(x)

    if isinstance(kernel, str):
        # Shared keyword arguments across dimensions
        kwargs = {
            'kernel': kernel,
            'sigma': sigma,
            'padding_type': padding_type,
            'antialiasing': antialiasing,
        }
        # Core resizing module
        x = resize_1d(x, -2, size=sizes[0], scale=scales[0], **kwargs)
        x = resize_1d(x, -1, size=sizes[1], scale=scales[1], **kwargs)
    elif isinstance(kernel, torch.Tensor):
        x = downsampling_2d(x, kernel, scale=int(1 / scale))

    x = reshape_output(x, b, c)
    x = cast_output(x, dtype)
    return x

if __name__ == '__main__':
    # Just for debugging
    torch.set_printoptions(precision=4, sci_mode=False, edgeitems=16, linewidth=200)
    a = torch.arange(64).float().view(1, 1, 8, 8)
    z = imresize(a, 0.5)
    print(z)
    #a = torch.arange(16).float().view(1, 1, 4, 4)

In [ ]:
resnet50 = models.resnet50(pretrained=True)  # download and load pretrained model
resnet50.eval()

# vgg16 = models.vgg16(pretrained=True)  # download and load pretrained model
# vgg16.eval()

In [ ]:
## mean and std will remain same irresptive of the model you use
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

preprocess11 = transforms.Compose([
    transforms.Normalize(mean, std)
])

labels_link = "https://savan77.github.io/blog/labels.json"
labels_json = requests.get(labels_link).json()
labels = {int(idx): label for idx, label in labels_json.items()}

## Set random seed for reproducibility
manualSeed = 999
# manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
padsize = 1
m = 224  #Pixel number of SLM
factor = 2  #For Nyquist sampling
factor2 = 1
M = m * factor * factor2  #Image size
f = 100000  #Focal length of relay lens in um
length = [0.61, 0.53, 0.47]  #Wavelength in um [Red Green Blue]. Note that center wavelegth is green color.
length = np.array(length)
fov = 250000  #Field-of-view in um
eff_pixelsize = fov/M  #Effective pixel size
slm_pixelsize = 30  #Pixel size of SLM in um
slm_pixelnum = m  #추가

PhySizeSLM = slm_pixelnum * slm_pixelsize
MaxFreqSLM = PhySizeSLM / length / f / 2
DelFreqSLM = slm_pixelsize / np.min(length) / f  #기준은 가장 High frequency를 전달할 수 있는 Blue channel

kx = (DelFreqSLM * torch.range(-M / 2, M / 2 - 1)).to(torch.double)
ky = (DelFreqSLM * torch.range(-M / 2, M / 2 - 1)).to(torch.double)
kxm, kym = torch.meshgrid(kx, ky)

Ratio = MaxFreqSLM / MaxFreqSLM[2]  #Radius ratio of each color channel

cent = M/2
initial = torch.rand(224, 224).to(device)

In [ ]:
## Random perturbation

class FourierDomain(torch.nn.Module):
    def __init__(self):
        super(FourierDomain, self).__init__()
        self.MM = torch.nn.Parameter(initial)
        self.MM.requires_grad = True

    def forward(self, inputIntensityFT, attack):
        slm = self.MM
        
        slm = torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(slm, 0), 0), scale_factor=factor2)))
        CTF_B = (torch.sqrt(torch.pow(kxm, 2) + torch.pow(kym, 2)) < MaxFreqSLM[2] * factor2).to(torch.double)
        CTF_B = CTF_B.to(device)
        slm = slm * CTF_B[int(cent-M/4):int(cent+M/4),int(cent-M/4):int(cent+M/4)]  #Physicially displayed SLM pattern
        
        slm_R = Ratio[0] * torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(slm, 0), 0), scale_factor=Ratio[0])))
        slm_G = Ratio[1] * torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(slm, 0), 0), scale_factor=Ratio[1])))
        slm_B = slm
        
        slm_r = torch.zeros(M, M).to(device)
        slm_g = torch.zeros(M, M).to(device)
        slm_b = torch.zeros(M, M).to(device)
        
        x, y = tuple(list(slm_R.size()))
        slm_r[int(M / 2 + 1 + floor(-y / 2)):int(M / 2 + 1 + floor(y / 2)),
        int(M / 2 + 1 + floor(-x / 2)):int(M / 2 + 1 + floor(x / 2))] = slm_R

        x, y = tuple(list(slm_G.size()))
        slm_g[int(M / 2 + 1 + floor(-y / 2)):int(M / 2 + 1 + floor(y / 2)),
        int(M / 2 + 1 + floor(-x / 2)):int(M / 2 + 1 + floor(x / 2))] = slm_G

        x, y = tuple(list(slm_B.size()))
        slm_b[int(M / 2 + 1 + floor(-y / 2)):int(M / 2 + 1 + floor(y / 2)),
        int(M / 2 + 1 + floor(-x / 2)):int(M / 2 + 1 + floor(x / 2))] = slm_B
        
        CTF_R = (slm_r != 0).to(torch.double)
        CTF_G = (slm_g != 0).to(torch.double)
        CTF_B = (slm_b != 0).to(torch.double)
        
        atk_CTF_R_mask_stack = torch.cat([torch.unsqueeze(torch.cos(slm_r), 2), torch.unsqueeze(torch.sin(slm_r), 2)],
                                         dim=2)
        atk_CTF_R_mask = torch.view_as_complex(atk_CTF_R_mask_stack)
        atk_CTF_R = CTF_R * atk_CTF_R_mask

        atk_CTF_G_mask_stack = torch.cat([torch.unsqueeze(torch.cos(slm_g), 2), torch.unsqueeze(torch.sin(slm_g), 2)],
                                         dim=2)
        atk_CTF_G_mask = torch.view_as_complex(atk_CTF_G_mask_stack)
        atk_CTF_G = CTF_G * atk_CTF_G_mask

        atk_CTF_B_mask_stack = torch.cat([torch.unsqueeze(torch.cos(slm_b), 2), torch.unsqueeze(torch.sin(slm_b), 2)],
                                         dim=2)
        atk_CTF_B_mask = torch.view_as_complex(atk_CTF_B_mask_stack)
        atk_CTF_B = CTF_B * atk_CTF_B_mask
        
        if attack == True:
            ft_atk_CTF_R = fftn(fftshift(atk_CTF_R))
            MTF_R = ifftshift(ifftn((ft_atk_CTF_R * torch.conj(ft_atk_CTF_R))))

            ft_atk_CTF_G = fftn(fftshift(atk_CTF_G))
            MTF_G = ifftshift(ifftn((ft_atk_CTF_G * torch.conj(ft_atk_CTF_G))))

            ft_atk_CTF_B = fftn(fftshift(atk_CTF_B))
            MTF_B = ifftshift(ifftn((ft_atk_CTF_B * torch.conj(ft_atk_CTF_B))))

        elif attack == False:
            ft_CTF_R = fftn(fftshift(CTF_R))
            MTF_R = ifftshift(ifftn((ft_CTF_R * torch.conj(ft_CTF_R))))

            ft_CTF_G = fftn(fftshift(CTF_G))
            MTF_G = ifftshift(ifftn((ft_CTF_G * torch.conj(ft_CTF_G))))

            ft_CTF_B = fftn(fftshift(CTF_B))
            MTF_B = ifftshift(ifftn((ft_CTF_B * torch.conj(ft_CTF_B))))

        MTF_R = MTF_R / torch.max(torch.max(torch.abs(MTF_R)))
        MTF_G = MTF_G / torch.max(torch.max(torch.abs(MTF_G)))
        MTF_B = MTF_B / torch.max(torch.max(torch.abs(MTF_B)))

        #inputIntensityFT = torch.as_tensor(inputIntensityFT, dtype=torch.cdouble)
        inputIntensityFT_R = inputIntensityFT[:, :, 0]
        inputIntensityFT_G = inputIntensityFT[:, :, 1]
        inputIntensityFT_B = inputIntensityFT[:, :, 2]
        
        outputFT_R = MTF_R * inputIntensityFT_R
        outputFT_G = MTF_G * inputIntensityFT_G
        outputFT_B = MTF_B * inputIntensityFT_B
        
        ifft_R = torch.abs(ifftn(ifftshift(outputFT_R))).to(torch.float32) / 255
        ifft_G = torch.abs(ifftn(ifftshift(outputFT_G))).to(torch.float32) / 255
        ifft_B = torch.abs(ifftn(ifftshift(outputFT_B))).to(torch.float32) / 255
                
        output_R = torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(ifft_R, 0), 0), size=(224, 224))))
        output_G = torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(ifft_G, 0), 0), size=(224, 224))))
        output_B = torch.squeeze(torch.squeeze(F.interpolate(torch.unsqueeze(torch.unsqueeze(ifft_B, 0), 0), size=(224, 224))))
    
        image_tensor2 = torch.stack([output_R, output_G, output_B])
        
    
        return image_tensor2, slm

In [ ]:
## find inputIntensityFT

img = Image.open(filepath)
tf = transforms.ToTensor()
IMG = tf(img)
IMG2 = imresize(IMG, sizes=(M, M))
IMG2 = IMG2.unsqueeze(0)
inputIntensityFT_R = fftshift(fftn(IMG2[:,0,:,:])).squeeze(0)
inputIntensityFT_G = fftshift(fftn(IMG2[:,1,:,:])).squeeze(0)
inputIntensityFT_B = fftshift(fftn(IMG2[:,2,:,:])).squeeze(0)
inputIntensityFT = torch.stack([inputIntensityFT_R, inputIntensityFT_G, inputIntensityFT_B])
inputIntensityFT = (inputIntensityFT * 255).permute(1, 2, 0)
inputIntensityFT = inputIntensityFT.detach().to(device)

In [ ]:
## original image(ground truth)

resnet50.to(device)
image_tensor = imresize(IMG, sizes=(224, 224))
image_tensor = image_tensor.detach().to(device)
image_tensor = preprocess11(image_tensor)  # preprocess an i
image_tensor = image_tensor.unsqueeze(0)  # add batch dimension.  C X H X W ==> B X C X H X W
output = resnet50.forward(image_tensor)
label_idx = torch.max(output.data, 1)[1][0]  # get an index(class number) of a largest element
label_idx = label_idx.item()
x_pred = labels[label_idx]
y_true = label_idx  ##change this if you change input image
target = Variable(torch.LongTensor([y_true]).to(device), requires_grad=False)

output_probs = F.softmax(output, dim=1)
x_pred_prob = np.round((torch.max(output_probs.data, 1)[0][0]).cpu() * 100, 4)
x_pred_prob = x_pred_prob.item()

print(x_pred)
print(x_pred_prob)

x = image_tensor.squeeze(0)
x = x.mul((torch.FloatTensor(std)).to(device).view(3, 1, 1)).add(
    (torch.FloatTensor(mean)).to(device).view(3, 1, 1)).cpu().numpy()  # reverse of normalization op- "unnormalize"
x = np.transpose(x, (1, 2, 0))  # C X H X W  ==>   H X W X C
x = np.clip(x, 0, 1)

plt.imshow(x)
plt.show()

In [ ]:
## Perturbation update

x_adv_pred = x_pred
param = 1e-2
n_epoch = 150
a = 0
i = 0

while x_adv_pred == x_pred:

    initial = torch.rand(224, 224).to(device)
    model = FourierDomain()
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=5 * (1e-3), weight_decay=5e-6)

    for epoch in range(0, n_epoch):
        i += 1
        optimizer.zero_grad()

        image_tensor2, slm = model(inputIntensityFT, attack=True)
        image_tensor2 = preprocess11(image_tensor2)  # preprocess an i
        image_tensor2 = image_tensor2.unsqueeze(0)
        output2 = resnet50.forward(image_tensor2)

        # perform a backward pass in order to get gradients 670 2000
        loss2 = param * torch.norm(image_tensor2 - image_tensor) - criterion(output2, target)
        
        x_adv_pred = labels[torch.max(output2.data, 1)[1][0].item()]  # classify adversarial example
        if x_adv_pred != x_pred:
            break

        loss2.backward()
        optimizer.step()
    
    if x_adv_pred != x_pred:
        break
        
    param /= 10
    a += 1

    if a == 5:
        break

##############################################################################

print('param: ', 1e-2 / (10**(i//150)), 'epoch: ', i%150)

In [ ]:
## adversarial image
output_adv_probs = F.softmax(output2, dim=1)
x_adv_pred_prob = np.round((torch.max(output_adv_probs.data, 1)[0][0]).cpu() * 100, 4)
x_adv_pred_prob = x_adv_pred_prob.item()

print(x_adv_pred)
print(x_adv_pred_prob)

x2 = image_tensor2.squeeze(0)
x2 = x2.mul((torch.FloatTensor(std)).to(device).view(3, 1, 1)).add(
    (torch.FloatTensor(mean)).to(device).view(3, 1, 1)).cpu().detach().numpy()  # reverse of normalization op- "unnormalize"
x2 = np.transpose(x2, (1, 2, 0))  # C X H X W  ==>   H X W X C
x2 = np.clip(x2, 0, 1)

plt.imsave(adv_filepath, x2)
plt.imshow(x2)
plt.show()

In [ ]:
## Image quality assessment

dist = torch.as_tensor(x2).permute(2, 0, 1).unsqueeze(0)
ref = torch.as_tensor(x).permute(2, 0, 1).unsqueeze(0)

model1 = SSIM(channels=3) # 0~1 값, 1에 가까울수록 원본과 유사도 높음
score1 = model1(dist, ref, as_loss=False)

model2 = VIF(channels=3) # 0~1 값, 1에 가까울수록 원본과 유사도 높음, 원본보다 이미지 퀄리티 좋을경우 1보다 클 수 있음
score2 = model2(dist, ref, as_loss=False)

print('SSIM of ResNet50: ', score1)
print('VIF of ResNet50: ', score2)

In [ ]:
## saving slm pattern
M = slm.cpu().detach().numpy()
df = pd.DataFrame(M)
df.to_excel(adv_filepath2, header=None, index=False)